# Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.io as spio
import matplotlib.pyplot as plt

from pymor.models.iosys import LTIModel

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Build model

Consider the cookies example in the state-space representation

\begin{align}
E \dot{x}(t) &= A x(t) + B u(t), \\
y(t) &= C x(t),
\end{align}

with $A, E \in \mathbb{R}^{n \times n}$, $B \in \mathbb{R}^{n \times m}$ and $C \in \mathbb{R}^{p \times n}$. 

In [ ]:
mat = spio.loadmat('data/ABCE.mat')
mat.keys()

In [ ]:
mu = np.sqrt(10) * np.array([0.2, 0.4, 0.6, 0.8])
A = mat['A0']
for i in range(4):
    A += mu[i] * mat[f'A{i + 1}']
B = mat['B']
C = mat['C']
E = mat['E']

In [ ]:
fom = LTIModel.from_matrices(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

# Bode plot

In [ ]:
w = np.logspace(-2, 4, 100)
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = fom.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = fom.mag_plot(w)

# Hankel singular values

Hankel singular values provide a joint measure for controllability and observability of the state variables in the balanced realization.

When computing Hankel singular values the solutions to the Lyapunov equations

\begin{align}
    A \mathcal{P} E^{\operatorname{T}} + E \mathcal{P} A^{\operatorname{T}} + B B^{\operatorname{T}} &= 0, \\
    A^{\operatorname{T}} \mathcal{Q} E + E^{\operatorname{T}} \mathcal{Q} A + C^{\operatorname{T}} C &= 0
\end{align}

are needed. The Hankel singular values are then given by

\begin{align}
    \sqrt{\Lambda(\mathcal{P} E^{\operatorname{T}} \mathcal{Q} E)}.
\end{align}

For high-dimensional systems in pyMOR the low-rank ADI iteration computes approximations $\mathcal{P} \approx Z_{\mathcal{P}} Z_{\mathcal{P}}^{\operatorname{T}}$ and $\mathcal{Q} \approx Z_{\mathcal{Q}} Z_{\mathcal{Q}}^{\operatorname{T}}$.

In [ ]:
hsv = fom.hsv()

In [ ]:
fig, ax = plt.subplots()
_ = ax.semilogy(range(1, len(hsv) + 1), hsv, '.-')

# Balanced truncation

1. Balance system w.r.t. to gramians $\mathcal{P}$ and $\mathcal{Q}$ which are solutions to

\begin{align}
    A \mathcal{P} E^{\operatorname{T}} + E \mathcal{P} A^{\operatorname{T}} + B B^{\operatorname{T}} &= 0, \\
    A^{\operatorname{T}} \mathcal{Q} E + E^{\operatorname{T}} \mathcal{Q} A + C^{\operatorname{T}} C &= 0.
\end{align}

2. Compute projectors $V$ and $W$ in order to truncate via Petrov-Galerkin projection.


3. Obtain error bound for free

\begin{equation}
	\lVert {H - \hat{H}} \rVert_{\mathcal{H}_{\infty}} \leq 2 \sum_{i = r+1}^{n} \sigma_i,
\end{equation}

where $\lVert {H - \hat{H}} \rVert_{\mathcal{H}_{\infty}} = \sup_{z \in \mathbb{C}_+} \lVert H(z) - \hat{H}(z)\rVert_2$.

In [ ]:
from pymor.reductors.bt import BTReductor

In [ ]:
bt = BTReductor(fom)

In [ ]:
rom = bt.reduce(r=5)

In [ ]:
fig, ax = plt.subplots()
error_bounds = bt.error_bounds()
_ = ax.semilogy(range(1, len(error_bounds) + 1), error_bounds, '.-')

In [ ]:
print(rom)

In [ ]:
rom.poles()

In [ ]:
rom.hinf_norm()

# Error system

The difference of transfer functions of two LTI systems

\begin{align}
H(s) - \hat{H}(s) = C (s E - A)^{-1} B - \hat{C} (s \hat{E} - \hat{A})^{-1} \hat{B}
\end{align}

can be represented as

\begin{align}
H(s) - \hat{H}(s) = \begin{bmatrix} C & -\hat{C} \end{bmatrix} \left(s \begin{bmatrix} E & 0 \\ 0 & \hat{E}\end{bmatrix} - \begin{bmatrix} A & 0 \\ 0 & \hat{A} \end{bmatrix} \right)^{-1} \begin{bmatrix} B \\ \hat{B}\end{bmatrix}.
\end{align}

In [ ]:
err = fom - rom

In [ ]:
err

In [ ]:
print(err)

# Bode plot of the error system

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = err.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = err.mag_plot(w)

# Related methods

- Balanced truncation variants:
  - LQG balanced truncation (`LQGBTReductor`)
  \begin{align*}
    &
    \begin{aligned}
      A^{\operatorname{T}} P E + E^{\operatorname{T}} P A - E^{\operatorname{T}} P B B^{\operatorname{T}} P E + C^{\operatorname{T}} C = 0
    \end{aligned} \\
    &
    A Q E^{\operatorname{T}} + E Q A^{\operatorname{T}}- E Q C^{\operatorname{T}} C Q E^{\operatorname{T}} + B B^{\operatorname{T}} = 0
  \end{align*} 
  - Balanced real balanced truncation (`BRBTReductor`)
  \begin{align*}
          A^T P E + E^T P A
          + (E^T P B) R^{-1} (E^T P B)^T
          + C^T C &= 0 \\
           A Q E^T + E Q A^T
          + (E Q C^T) R^{-1} (E Q C^T)^T
          + B B^T &= 0 \\
  \end{align*}
  
- Balanced truncation for second order systems:
    - `pymor.reductors.sobt` provides `SOBTpReductor`, `SOBTvReductor`, ...
